In [1]:
import torch

import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import random_split, DataLoader
from pytorch_lightning.loggers import TensorBoardLogger


from data_processing import portraits_dataset, load
import resnet 


In [2]:
hyperparameters = { 'batch size': 15,
                    'val ratio': 0.1,
                    'epochs': 35,
                    'lr': 0.0000095,
                    'lr decay': 0.25,
                    'lr decay threshold': 0.05,
                    'lr warming up period': 50,
                    'weight decay': 0.01}

In [3]:
class CrusaderPaints(pl.LightningModule):

    def __init__(self, model):
        super(CrusaderPaints, self).__init__()
        self.model = model
        self.learning_rate = hyperparameters['lr']
        self.weight_decay = hyperparameters['weight decay']


    def mse(self, x, y):
        return  F.mse_loss(x, y)


    def training_step(self, batch, batch_idx):
        image, dna = batch
        image = image.view(-1, 6, 224, 224)
        pre_dna = self.model(image)
        post_dna = torch.squeeze(torch.stack([dna for _ in range(pre_dna.size(dim=0))]))

        loss = self.mse(pre_dna, post_dna)       
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        image, dna = batch
        image = image.view(-1, 6, 224, 224)
        pre_dna = self.model(image)
        post_dna = torch.squeeze(torch.stack([dna for _ in range(pre_dna.size(dim=0))]))


        loss = self.mse(pre_dna, post_dna)
        return {'val_loss': loss}

    def trainning_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        self.log('loss', torch.sqrt(avg_loss), prog_bar=True)
        return {'loss': avg_loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        self.log('val_loss', torch.sqrt(avg_loss), prog_bar=True)
        return {'val_loss': avg_loss}

   
    def configure_optimizers(self):
        optimizer = torch.optim.RMSprop(self.model.parameters(), lr = self.learning_rate, weight_decay = self.weight_decay)
        return optimizer       

                                                   

In [4]:
class Paintings(pl.LightningDataModule):
    def __init__(self, batch_size, val_ratio, dataset_path):
        self.batch_size = batch_size
        self.val_ratio = val_ratio
        self.dataset_path = dataset_path

    def setup(self, stage=None):
        inputs, targets = load(self.dataset_path)
        portraits = portraits_dataset(inputs, targets)
        val_value = int(self.val_ratio*portraits.n_samples)
        self.portraits_train, self.portraits_test = random_split(portraits, [len(portraits)-val_value, val_value])
        
    def train_dataloader(self):
        return DataLoader(self.portraits_train, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.portraits_test, batch_size=self.batch_size)

In [5]:
data_module = Paintings(hyperparameters['batch size'], hyperparameters['val ratio'], 'datasets/dataset.pkl')
model = CrusaderPaints(resnet.ResNet50(img_channel=6, num_features=101))
trainer = Trainer(gpus=1, max_epochs=hyperparameters['epochs'])

c:\Users\hakim\Documents\Python\CK3 Portrait Generator\resnet.py:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(layer)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [6]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 23.7 M
---------------------------------
23.7 M    Trainable params
0         Non-trainable params
23.7 M    Total params
94.898    Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\hakim\Deeplearning\lib\site-packages\pytorch_lightning\trainer\data_loading.py:116: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


RuntimeError: CUDA out of memory. Tried to allocate 414.00 MiB (GPU 0; 4.00 GiB total capacity; 1.99 GiB already allocated; 27.08 MiB free; 2.44 GiB reserved in total by PyTorch)

[]
